In [ ]:
%pylab notebook
from __future__ import print_function 
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 720)
pd.set_option('display.max_columns', 30)

In [ ]:
%cd /Users/brodzik/2017/AGU_Hunza_melt_onset/
%ls

# Read spreadsheets and clean up headers

In [ ]:
station = "Khunjerab"
#station = "Ziarat"
if "Khunjerab" == station:
    file = 'Khunjerabnear_uib50-263.working.xlsx'
    station_elevation_m = 4440
    df = pd.read_excel(open(file,'rb'), sheetname='pixelUIB50-263glnearKh')
    melt_onset = '2003-04-12'
    xpgr_range = [-0.05,0.4]
elif "Ziarat" == station:
    file = 'gl_near_ziaratandmet.working.xlsx'
    station_elevation_m = 3020
    df = pd.read_excel(open(file,'rb'), sheetname='pixel_near_ziaratTb36V2003SIR')
    melt_onset = '2003-04-07'
    xpgr_range = [-0.02,0.2]

# Drop the 1st 2 rows, we don't need them
df = df.drop([0,1])

# Move the contents of data row 2 into the column headers
# and then drop row 2
df.columns = df.loc[2].values
df = df.drop([2])

# Move the df index to the Date_time_utc column
df = df.set_index('Date_time_utc')
df

# Read the Hunza daily discharge data

In [ ]:
dischargeFile = '/Users/brodzik/projects/CHARIS/streamflow/Pakistan/Hunza/hunza_at_dainyor_bridge_daily_timeseries_2003.txt'
hunza = pd.read_csv(dischargeFile, header=None, sep='\s+',
                    names=['year', 'month', 'day', 'doy', 'Q', 'dummy'])

hunza['Date'] = pd.to_datetime(hunza.year*10000 + hunza.month*100 + hunza.day,format='%Y%m%d')
hunza.set_index('Date', drop=True, inplace=True)
hunza.drop(['year','month','day', 'doy', 'dummy'], axis=1, inplace=True)
hunza

In [ ]:
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(3, 1, height_ratios=[3,3,1])

#fig, ax = plt.subplots(3, 1, figsize=(8,8))
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])
ax2 = plt.subplot(gs[2])
line_36v = df["TB_36V_K"].plot(ax=ax0, color='mediumslateblue', 
                               label='$T_B$ 36V',
                               marker='.', linestyle=':')
line_18h = df["TB_18H_K"].plot(ax=ax0, color='peru',
                               label='$T_B$ 18H',
                               marker='.', linestyle=':')
ax0.set_title('Glacier near %s (%d m): $T_B$ and XPGR (2003)' % (
    station, station_elevation_m))
ax0.set_ylim([150., 300.])
ax0.set_xlabel("Date")
ax0.set_ylabel("$T_B$ ($K$)")
ax0.grid(True)

gridlines = ax0.get_xgridlines() + ax0.get_ygridlines()
for line in gridlines:
    line.set_color('lightgray')
    line.set_linestyle(':')

# Use a different scale for xpgr trace
ax0R = ax0.twinx()
line_xpgr = df["XPGR"].plot(ax=ax0R, color='gray', label='XPGR')
ax0R.set_ylim(xpgr_range)
ax0R.set_ylabel('XPGR', color='gray')
ax0R.tick_params('y', colors='gray')

# Plot likely melt onset date
line_v = ax0R.axvline(pd.to_datetime(melt_onset), 
                      label='$T_{max} > 0^o C$',
                      color='k', linestyle=':', linewidth=3.0)

# Combine the legends
leftlines, leftlabels = ax0.get_legend_handles_labels()
rightlines, rightlabels = ax0R.get_legend_handles_labels()
lines = leftlines + rightlines
labels = leftlabels + rightlabels

ax0.legend(lines, labels, loc='upper right')

# Work on middle plot
#line_36v = df["TB_36V_K"].plot(ax=ax1, color='mediumslateblue', 
#                               label='$T_B$ 36V',
#                               marker='.', linestyle=':')
#line_18h = df["TB_18H_K"].plot(ax=ax1, color='peru',
#                               label='$T_B$ 18H',
#                               marker='.', linestyle=':')
ax1.set_title('Glacier near %s (%d m): DAV and Temperature' % (
    station, station_elevation_m))
ax1.set_ylim([-30, 60.])
ax1.set_xlabel("Date")
ax1.set_ylabel("Degrees")
ax1.grid(True)

gridlines = ax1.get_xgridlines() + ax1.get_ygridlines()
for line in gridlines:
    line.set_color('lightgray')
    line.set_linestyle(':')
    
# Use degrees (C or K) for right hand axis
#ax1R = ax1.twinx()
#ax1R.set_ylim([-30., 100.])
#ax1R.set_ylabel('Degrees', color='gray')
#ax1R.tick_params('y', colors='gray')


# Use right hand scale for |DAV|  ---OBE
line_dav_36v = df["Abs_DAV_36V"].plot(ax=ax1, 
                                      color='mediumslateblue',
                                      label='|$DAV_{36V}$|')
line_dav_18h = df["Abs_DAV_18H"].plot(ax=ax1, 
                                      color='peru',
                                      label='|$DAV_{18H}$|')

line_tmax = ax1.plot(df.doy, df.T_max_C, color='r', label="T_max")
line_tmin = ax1.plot(df.doy, df.T_min_C, color='b', label="T_min")

# Add a line for freezing surface temperature
ax1.axhline(color='r', linestyle=":")

# Plot likely melt onset date
line_v = ax1.axvline(pd.to_datetime(melt_onset), 
                      label='$T_{max} > 0^o C$',
                      color='k', linestyle=':', linewidth=3.0)

# Combine the legends
#leftlines, leftlabels = ax1.get_legend_handles_labels()
#rightlines, rightlabels = ax1R.get_legend_handles_labels()
#lines = leftlines + rightlines
#labels = leftlabels + rightlabels
#ax1.legend(lines, labels, loc="upper right")
ax1.legend(loc="upper right")
 
# Third plot is Hunza discharge
line_Q = hunza["Q"].plot(ax=ax2, color='k', label='Hunza at Dainyor Bridge')
ax2.set_title('Measured discharge, Hunza at Dainyor Bridge, 2003')
ax2.set_ylabel('Q ($m^3 s^{-1}$)')
              
fig.tight_layout()
outfile = "/Users/brodzik/2018/EGU/%s_XPGR_DAV_2003_tseries.pdf" % station
plt.savefig(outfile)
print("Image saved to %s" % outfile)

# Now plot min/max station temperature vs. morning/evening CETB

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

# v1.2 correction: M and E are mislabelled
# if we repeat this with v1.3, these will be shifted
#evening_TB_K = df.TB_36V_midday
#morning_TB_K = df.TB_36V_night

if 'Khunjerab' == station:
    morning_TB_K = df['TB_36V_K'].values[1::2]
    evening_TB_K = df['TB_36V_K'].values[::2]
elif 'Ziarat' == station:
    morning_TB_K = df['TB_36V_night'].values[:365]
    evening_TB_K = df['TB_36V_midday'].values[:365]
    
morning_station_C = df['T_min_C'].values[:365]
evening_station_C = df['T_max_C'].values[:365]

# T_min_C compared to nighttime TB
line_morning = plt.scatter(morning_station_C, morning_TB_K,
                           color='b',
                           label="Station $T_{min}$ vs. Morning $T_B$")
line_evening = plt.scatter(evening_station_C, evening_TB_K,
                           color='r',
                           label="Station $T_{max}$ vs. Evening $T_B$")
ax.set_title('Surface Temperature vs. AMSRE-E near %s (2003)' % station)
ax.set_xlim([-30., 30.])
ax.set_xlabel("Station Temperature ($^oC$)")
ax.set_ylabel("37 Vertically-Polarized $T_B$ ($K$)")
ax.grid(True)

gridlines = ax.get_xgridlines() + ax.get_ygridlines()
for line in gridlines:
    line.set_color('lightgray')
    line.set_linestyle(':')

ax.legend(loc='lower right')

   
fig.tight_layout()
outfile = "/Users/brodzik/2018/EGU/%s_Surface_vs_TB_2003.pdf" % station
plt.savefig(outfile)
print("Image saved to %s" % outfile)

In [ ]:
df.T_min_C.values[:365]


In [ ]:
df['TB_36V_midday'].values[:365]

In [ ]:
morning

In [ ]:
evening

In [ ]:
df